### Setup

First, we need to install the relevant libaries required for this local training usecase. You will need to pip install boto3, pandas and Sagemaker into your Python environment. For this lab, running Python 3.9 as your kernel is recommended. Please see the requirements.txt for what's needed.


In [ ]:
import tarfile
import boto3
import pandas as pd
import os
from sagemaker.estimator import Estimator
from sagemaker.local import LocalSession
from sagemaker.predictor import csv_serializer

In [ ]:
from __future__ import print_function

import json
import os
import pickle
import sys
import traceback

import pandas as pd
from causalnex.discretiser import Discretiser
import warnings
from causalnex.structure import StructureModel
from sklearn.model_selection import train_test_split
from causalnex.network import BayesianNetwork
from causalnex.evaluation import classification_report
from causalnex.evaluation import roc_auc

The next step is to create a SageMaker Local session. Please insert the ARN for Sagemeker execution role below. 

*Note*

*You will need to create an IAM role in your AWS account and ensure it has permissions to SageMaker, S3 and ECR.*


In [ ]:
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

role = 'arn:aws:iam::403775705461:role/SageMaker-IAM-Role-AB3'

Define the data location, which is apart of this repository.


In [ ]:
data_location = "./data/heart_failure_clinical_records_dataset.csv"

### Data Pre-processing

In the cell, we are doing data pre-processing to make our dataset ML friendly. This code is using a Discretiser class from the causalnex library to transform a continuous feature into a discrete one. 

Taking age as an example, we are using an numeric_split_points=[60], which means it will split the data into two bins: below 60 and above 60. Similar approaches are used on other columns.

In [ ]:
from causalnex.discretiser import Discretiser
import pandas as pd

initial_df = pd.read_csv(data_location)

initial_df["age"] = Discretiser(method="fixed", numeric_split_points=[60]).transform(
    initial_df["age"].values
)
initial_df["serum_sodium"] = Discretiser(method="fixed", numeric_split_points=[136]).transform(
    initial_df["serum_sodium"].values
)
initial_df["serum_creatinine"] = Discretiser(
    method="fixed", numeric_split_points=[1.1, 1.4]
).transform(initial_df["serum_sodium"].values)

initial_df["ejection_fraction"] = Discretiser(
    method="fixed", numeric_split_points=[30, 38, 42]
).transform(initial_df["ejection_fraction"].values)

initial_df["creatinine_phosphokinase"] = Discretiser(
    method="fixed", numeric_split_points=[120, 540, 670]
).transform(initial_df["creatinine_phosphokinase"].values)

initial_df["platelets"] = Discretiser(method="fixed", numeric_split_points=[263358]).transform(
    initial_df["platelets"].values
)

print ("Dataset after pre-processing")
initial_df.head()

### Training

In the next cell, we will be training the model on our dataset from above. You should see training information when the training is complete.

In [ ]:
sm = StructureModel()
sm.add_edges_from([
    ('ejection_fraction', 'DEATH_EVENT'),
    ('creatinine_phosphokinase', 'DEATH_EVENT'),
    ('age','DEATH_EVENT'),
    ('smoking','high_blood_pressure'),
    ('age','high_blood_pressure'),            
    ('serum_sodium','DEATH_EVENT'),
    ('high_blood_pressure','DEATH_EVENT'),
    ('anaemia','DEATH_EVENT'),
    ('creatinine_phosphokinase','DEATH_EVENT'),
    ('smoking','DEATH_EVENT')
])

train, test = train_test_split(initial_df, train_size=0.8, test_size=0.2, random_state=42)
        
bn = BayesianNetwork(sm)
bn = bn.fit_node_states(initial_df)
bn = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")

roc, auc = roc_auc(bn, test, "DEATH_EVENT")
print("Model AUC: " + str(auc))

print(classification_report(bn, test, "DEATH_EVENT"))

# save the model
model_path = "models"
isExist = os.path.exists(model_path)
if not isExist:
   os.makedirs(model_path)
with open(os.path.join(model_path, 'causal_model.pkl'), 'wb') as out:
    pickle.dump(bn, out)

### Deploy the Docker Container Locally

For this step, we need to create a docker image in our local environment. Please ensure you have 'sagemaker-causalnex-local' in your Docker Images. 

1. Please ensure Docker is running locally
2. Please run 'docker build -t sagemaker-causal-nex:latest .' in the 'sagemaker-local-to-cloud/local/container' path
3. From the previous steps, please insert the IAM role from your AWS account with permissions to access SageMaker, S3 and ECR and insert the ARN in the 'role_value' section

In [ ]:
# needs to change to ../local/container/Dockerfile when finished
! docker build -f ../local/container_SM_Toolkit/Dockerfile -t sagemaker-causal-nex:latest .

In [ ]:
from sagemaker.local import LocalSession

image = 'sagemaker-causal-nex'

env={
    "MODEL_SERVER_WORKERS":"2"
    }

local_regressor = Estimator(
    image,
    role = 'arn:aws:iam::403775705461:role/SageMaker-IAM-Role-AB3',
    train_instance_count=1,
    train_instance_type="local")

train_location = 'file://'+data_location

local_regressor.fit(train_location, logs=True)

Now we can launch this container and run it!

In [ ]:
predictor = local_regressor.deploy(1, 'local', env=env)

### Inference

We can now send a sample JSON payload for inference with our model. 

In [ ]:
test_data = open('payload.json')
test_data1 = '{"age": 1, "anaemia": 0, "creatinine_phosphokinase": 2, "diabetes": 0, "ejection_fraction": 0, "high_blood_pressure": 1, "platelets": 1, "serum_creatinine": 0, "serum_sodium": 0, "sex": 1, "smoking": 0, "time": 4}'

#with open('payload.json') as f:
#    d = json.load(f)
#    s = json.dumps(d)
#    print(d)

with open('payload.json') as f:
    test = json.load(f)


# s = json.dumps(test_data1)
print (test)
print(type(test)) 
# print (s)

In [ ]:
predicted = predictor.predict(test["data"]).decode('utf-8')
# predicted = predictor.predict(test["data"]).decode('utf-8')

# predicted = predictor.predict(s)

In [ ]:
print(predicted)

### Push the Container to ECR on the AWS Cloud

At this Point, we have successfully launched the container on our local machine and we are able to send inference commands. We would now like to push this container to the ECR repository on the AWS Cloud.  

We start by defining some variables like the current execution role, the ECR repository that we are going to use for pushing the custom Docker container and a default Amazon S3 bucket to be used by Amazon SageMaker.

In [ ]:
import sagemaker

ecr_namespace = "sagemaker-local-training-containers/"
prefix = "local-training"

ecr_repository_name = ecr_namespace + prefix
account_id = role.split(":")[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)


Let's take a look at the Dockerfile which defines the statements for building our custom SageMaker training container:


In [ ]:
! pygmentize ../local/container/Dockerfile

At high-level the Dockerfile specifies the following operations for building this container:

TODO

### Build and push the container
We are now ready to build this container and push it to Amazon ECR. It will create a new repo in ECR for you. Please ensure you have the correct IAM permissions to push to the ECR.

In the below cell we are building, tagging, authenticating and pushing the container to ECR with the above variables.

In [ ]:
! docker build -f ../local/container/Dockerfile -t sagemaker-local-training-containers/tutorial ../local/container
! docker tag sagemaker-local-training-containers/tutorial {account_id}.dkr.ecr.{region}.amazonaws.com/sagemaker-local-training-containers/local-training:latest
! aws ecr get-login --no-include-email --registry-ids {account_id}
! aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
! aws ecr describe-repositories --repository-names sagemaker-local-training-containers/local-training || aws ecr create-repository --repository-name sagemaker-local-training-containers/local-training
! docker push {account_id}.dkr.ecr.{region}.amazonaws.com/sagemaker-local-training-containers/local-training:latest



Alternaitvely you can use the below cell and leverage the python script

In [ ]:
# %%capture
# ! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name

Shut down the endpoint we created for this tutorial.

In [ ]:
predictor.delete_endpoint()